# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 24.4 MB/s eta 0:00:00


In [57]:
from ultralytics import YOLO, solutions
import cv2

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [58]:
model_id = '/content/YOLO_Model.pt'
model = YOLO(model_id)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [59]:
cap = cv2.VideoCapture("/content/YOLO_Video.mp4")
assert cap.isOpened(), "Error reading video file"

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [60]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [61]:
#region_points
mid_x = w // 2
mid_y = h // 2

r1 = [(10,810), (mid_x- 10 ,810)]
r2 = [(1440,540),(1440,h)]
r3 = [(480, 10),(480, 530)]
r4 = [(mid_x + 10,270),(w-10,270)]

In [62]:
counter1 = solutions.ObjectCounter(
    view_img=True,
    reg_pts= r1,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,)

counter2 = solutions.ObjectCounter(
    view_img=True,
    reg_pts= r2,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,)

counter3 = solutions.ObjectCounter(
    view_img=True,
    reg_pts= r3,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,)

counter4 = solutions.ObjectCounter(
    view_img=True,
    reg_pts= r4,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,)

classes_to_count = [1,2]

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.


In [63]:
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [64]:
while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video frame is empty or video processing has been successfully completed.")
            break
        tracks = model.track(im0, persist=True, show=False, classes=classes_to_count)


        im0_r1 = counter1.start_counting(im0.copy(), tracks)
        im0_r2 = counter2.start_counting(im0.copy(), tracks)

        im0_top = cv2.addWeighted(im0_r1, 0.5, im0_r2, 0.5, 0)

        im0_r3 = counter3.start_counting(im0.copy(), tracks)
        im0_r4 = counter4.start_counting(im0.copy(), tracks)

        im0_bottom = cv2.addWeighted(im0_r3, 0.5, im0_r4,0.5, 0)

        im0_all = cv2.addWeighted(im0_top, 0.5, im0_bottom, 0.5, 0)


        bottom_count = counter1.count_ids
        top_count =counter4.count_ids

        right_count = counter2.count_ids
        left_count = counter3.count_ids

        text = f"Left exit(W): {len(left_count)} | Right exit(E): {len(right_count)} | top exit (N): {len(top_count)} | bottom exit (S): {len(bottom_count)}"
        cv2.putText(im0_all, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


        video_writer.write(im0_all)


0: 384x640 82 cars, 2 trucks, 64.1ms
Speed: 3.1ms preprocess, 64.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 2 trucks, 43.7ms
Speed: 3.1ms preprocess, 43.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 2 trucks, 41.6ms
Speed: 3.1ms preprocess, 41.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 2 trucks, 41.6ms
Speed: 3.2ms preprocess, 41.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 2 trucks, 41.5ms
Speed: 2.1ms preprocess, 41.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 2 trucks, 41.6ms
Speed: 2.5ms preprocess, 41.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 2 trucks, 41.6ms
Speed: 3.2ms preprocess, 41.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 84 cars, 2 trucks, 41.6ms
Speed: 3.1ms prep

In [65]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

In [66]:
import subprocess

def convert_avi_to_mp4(avi_file, mp4_file):

    command = ['ffmpeg', '-i', avi_file, '-codec', 'copy', mp4_file]
    subprocess.run(command, check=True)

avi_file = '/content/object_counting_output.avi'
mp4_file = '/content/output_video.mp4'

try:
    convert_avi_to_mp4(avi_file, mp4_file)
    print(f"Successfully converted {avi_file} to {mp4_file}")
except subprocess.CalledProcessError as e:
    print(f"Error during conversion: {e}")

Successfully converted /content/object_counting_output.avi to /content/output_video.mp4


Submit both the notebook and the output video

## I will upload the output video in github. just in case, i will upload it again on [my drive](https://drive.google.com/drive/folders/16OGxpZEXz-QIs2LDQrr4ZtXaARF56amt?usp=drive_link)


